In [4]:
import matplotlib.pyplot as plt
import numpy as np

from tutils import BaseStateSystem

#######################################
# Laplacian + initialisation functions#
#######################################

def laplacian1D(a, dx):
    return (
        - 2 * a
        + np.roll(a,1,axis=0) 
        + np.roll(a,-1,axis=0)
    ) / (dx ** 2)

def random_initialiser(shape):
    return(
        np.random.normal(loc=0, scale=0.05, size=shape),
        np.random.normal(loc=0, scale=0.05, size=shape)
    )

def function_initialiser(shape,f1,f2):
    space = range(shape)
    return(
        np.fromiter(map(f1,space),float),
        np.fromiter(map(f2,space),float)
    )

# Initialisation functions
f1 = lambda x : eq_pt*(1 + pert_a(x))
f2 = lambda x : eq_pt*(1 + pert_b(x))

#######################################
# eq_point, and perturbation choices  #
#######################################

# constants used to define wavenumber
m_vals = [7, 12, 23, 29]
m = m_vals[0]

# Perturbation from equilibrium
def pert_a(x): return np.cos(m*2*np.pi*x/100)
def pert_b(x): return np.cos(m*2*np.pi*x/100)

# Reaction equations
def Ra(a,b): return a - a ** 3 - b + alpha
def Rb(a,b): return (a - b) * beta

# Diffusion and reaction constants 
Da, Db, alpha, beta = 1, 100, -0.005, 10

# Computing the equilibrium point of the system. 
eq_pt = np.cbrt(alpha)

#######################################################
# Main class #
#######################################################

class OneDimensionalRDEquations(BaseStateSystem):
    def __init__(self, Da, Db, Ra, Rb,
                 initialiser=random_initialiser,
                 width=1000, dx=1, 
                 dt=0.1, steps=1):
        
        self.Da = Da
        self.Db = Db
        self.Ra = Ra
        self.Rb = Rb
        
        self.initialiser = initialiser
        self.width = width
        self.dx = dx
        self.dt = dt
        self.steps = steps
 
    def initialise(self):
        self.t = 0
        self.a, self.b = function_initialiser(self.width,f1,f2)

    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def _update(self):
        # unpack so we don't have to keep writing "self"
        a,b,Da,Db,Ra,Rb,dt,dx = (
            self.a, self.b,
            self.Da, self.Db,
            self.Ra, self.Rb,
            self.dt, self.dx
        )

        La = laplacian1D(a, dx)
        Lb = laplacian1D(b, dx)
        
        delta_a = dt * (Da * La + Ra(a,b))
        delta_b = dt * (Db * Lb + Rb(a,b))

        self.a += delta_a
        self.b += delta_b
        
    def draw(self, ax):
        print ("draw: {}".format(self.t))
        ax.clear()
        ax.plot(self.a, color="r", label="A")
        ax.plot(self.b, color="b", label="B")
        ax.legend()
        ax.set_ylim(-1, 1)
        ax.set_title("t = {:.2f}".format(self.t))

    def custom_draw(self, vals, title, filename):
        fig, ax = self.initialise_figure()
        # accurate x-axis markers
        x = np.linspace(0, self.steps*dt*len(vals), len(vals))
        ax.clear()
        ax.plot(x, vals, color="r", label="a")
        plt.xlabel('t')
        ax.legend()
        #ax.set_ylim(min(vals), max(vals))
        ax.set_ylim(-3, 3)
        fig.savefig('plots/' + filename + '.png')
        plt.close()
    
    # plots the amplitude of a(x,t) over n_steps as a function of t by taking
    # the ptp (peak to peak) value of a(x,t) at each time t and plotting the result. 
    def plot_amplitude(self, n_steps):
        self.initialise() # reset equation before plotting
        amp = np.array([])

        for _ in range(n_steps):
            amp = np.append(amp, self.a.ptp()) # measure peak to peak, i.e., get amplitude
            self.update()

        title = "amp,t={0:.2f},n={1}".format(self.t, n_steps)
        filename = "amp_plot_n={}".format(n_steps)
        self.custom_draw(amp, title, filename)

    # tracks a(pt, t) for n_steps of time, t. Plots a as a function of t.
    def track_growth(self, pt, n_steps):
        self.initialise() # reset equation before tracking
        orbit_a = np.array([])

        for _ in range(n_steps):
            orbit_a = np.append(orbit_a, self.get_val(pt))
            self.update()
            
        return orbit_a

    def plot_growth(self, pt, n_steps):
        orbit_a = self.track_growth(pt, n_steps)
        title = "growth,t={0:.2f},n={1},pt={2}".format(self.t, n_steps, pt)
        filename = "growth_x={0:.1f}_n={1}".format(pt, n_steps)
        self.custom_draw(orbit_a, title, filename)

    def approx_omega(self, growth):
        return (np.log(np.abs( growth/eq_pt - 1) ))

    def plot_growth_rate(self, pt, n_steps):
        growth = self.track_growth(pt, n_steps)
        growth_line = self.approx_omega(growth)

        title = "growth_rate_t={0:.2f}_n={1}_pt={2}".format(self.t, n_steps, pt)
        filename = "growth_x={0:.1f}_n={1}_m={2}".format(pt, n_steps, m)
        self.custom_draw(growth_line, title, filename)

    def get_val(self, pt):
        assert 0 <= pt < self.width, "Point is not in the domain"
        if pt in range(self.width):
            return self.a[pt]
        else: 
            return self.interpolate(pt)
        
    def interpolate(self, pt):
        x_vals = [int(np.floor(pt)), int(np.ceil(pt))]
        a_vals = [self.a[j] for j in x_vals]
        return np.interp(pt, x_vals, a_vals)

# Local playground values #

width = 100
steps = 1
dx = 1
dt = 0.001

equation = OneDimensionalRDEquations(
    Da, Db, Ra, Rb, 
    width=width, dx=dx, dt=dt, 
    steps=steps
)

# Pre-initialise our equation so we don't have to keep calling this in playground
equation.initialise()



None


Plotting the time evolution for m = 29. 

![SegmentLocal](gifs/1dRD_m=29.gif "segment") 

Plotting time evolution for m = 7.

![SegmentLocal](gifs/1dRD_m=7.gif "segment") 

Fix a particular x and track the growth rate of **u**(x,t). Here for x = 0, and x = 7.1

A | B
- | - 
![alt](plots/growth_x=0.0_n=80.png) | ![alt](plots/growth_x=7.1_n=80.png)

It looks roughly exponential! We investigate if the the numerical simulations align with the predicted growth rates in the theoretical analysis. We'll use our log formula to approximate omega. In particular, for m=7 we observe the following graph. 

![alt](plots/predic_vs_sim_m=7.png)

Positive predicted growth rates compared with our Log approximation.

![alt](plots/pred_vs_sim_pos_m.png)

Negative predicted growth rates compared with our Log approximation.

A | B
- | - 
![alt](plots/pred_vs_sim_m_neg.png) | ![alt](plots/bad_bois.png)